In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 20,12
import warnings
warnings.filterwarnings('ignore')
# Configuration
pd.set_option('max_column', None)
colors = sns.color_palette('Set2')
from tqdm.notebook import tqdm
tqdm.pandas()
from decimal import ROUND_HALF_UP, Decimal
from datetime import datetime, timedelta
ix = pd.IndexSlice
!pip install arch
import arch

path = 'C:/Users/shubh/Desktop/Python And Finance/jpx-tokyo-stock-exchange-prediction/train_files'

In [3]:
#load stocks info
stocks = pd.read_csv(f'{path}/stock_list.csv')
stocks['SecuritiesCode'] = stocks['SecuritiesCode'].astype(int)
stocks = stocks[stocks['Universe0']==True]
stocks = stocks.iloc[:]

#
codes = stocks['SecuritiesCode'].values

In [4]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")
    price['SecuritiesCode'] = price['SecuritiesCode'].astype(int)

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (df["CumulativeAdjustmentFactor"] * df["Close"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)))
        # reverse order
        df = df.sort_values("Date")
        # Generate cumulative adjustment factor volume
        df.loc[:, 'CumAdjustmentFactorVol'] = df['AdjustmentFactor'].cumprod()
        
        # generate AdjustedVolume 
        df['AdjustedVolume'] = df['Volume'].where(df['AdjustmentFactor'] != 1, (df['Volume'] * df['CumAdjustmentFactorVol']).astype('int64'))

        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        df.loc[df['AdjustedVolume']==0,'AdjustedVolume'] = np.nan
        df.loc[:, "AdjustedVolume"] = df.loc[:, "AdjustedVolume"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)
    

    price.set_index("Date","SecuritiesCode",inplace=True)
    return price

In [65]:
# load stock price data
df_price = pd.read_csv(f"{path}/stock_prices.csv")

# generate AdjustedClose
df_price = adjust_price(df_price)

In [66]:
df_price

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CumulativeAdjustmentFactor,AdjustedClose,CumAdjustmentFactorVol,AdjustedVolume
Date,,,,,,,,,,,,,,,
2017-01-04,20170104_1301,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,1.0,2742.0,1.0,31400.0
2017-01-05,20170105_1301,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,1.0,2738.0,1.0,17900.0
2017-01-06,20170106_1301,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,1.0,2740.0,1.0,19900.0
2017-01-10,20170110_1301,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,1.0,2748.0,1.0,24200.0
2017-01-11,20170111_1301,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,1.0,2745.0,1.0,9300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-29,20211129_9997,9997,678.0,679.0,665.0,668.0,320800,1.0,NaN,False,0.026987,1.0,668.0,1.0,320800.0
2021-11-30,20211130_9997,9997,670.0,689.0,667.0,667.0,296300,1.0,NaN,False,-0.001460,1.0,667.0,1.0,296300.0
2021-12-01,20211201_9997,9997,661.0,688.0,660.0,685.0,339100,1.0,NaN,False,0.017544,1.0,685.0,1.0,339100.0


In [67]:
# technical indicators
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

def rsi_class(x):
    ret = "low"
    if x < 50:
        ret = "low"
    if x > 50:
        ret = "med"
    if x > 70:
        ret = "hi"
    return(ret)

def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code, ["SecuritiesCode","Target", close_col,"AdjustedVolume"]].copy()

    # calculate 2 week return using AdjustedClose
    feats["return_2week"] = feats[close_col].pct_change(10)
    # calculate last 1 month return using AdjustedClose
    feats["return_1month"] = feats[close_col].pct_change(21)
    # calculate last 3 months return using AdjustedClose
    feats["return_3month"] = feats[close_col].pct_change(63)

    # calculate 2 week historical volatility using AdjustedClose
    feats["volatility_2week"] = (np.log(feats[close_col]).diff().rolling(10).std())
    # calculate last 1 month historical volatility using AdjustedClose
    feats["volatility_1month"] = (np.log(feats[close_col]).diff().rolling(21).std())
    # calculate last 3 months historical volatility using AdjustedClose
    feats["volatility_3month"] = (np.log(feats[close_col]).diff().rolling(63).std())

    # RSI
    for period in [10, 21, 63]:
        feats[f"feature_RSI_{period}days"] = RSI(feats[close_col], period)
        #feats[f"rsi{period}"] = list(map(rsi_class, feats[f"feature_RSI_{period}days"] 
        #feats[f"rsi{period}"] = pd.Series(feats[f"rsi{period}"], dtype="category")
    feats['rsi10'] = list(map(rsi_class, feats['feature_RSI_10days']))
    feats['rsi21'] = list(map(rsi_class, feats['feature_RSI_21days']))
    feats['rsi63'] = list(map(rsi_class, feats['feature_RSI_63days']))
    #moving averages
    for period in [10, 21, 63]:
        feats[f"EMA{period}days"] = feats[close_col].ewm(span=period,adjust=False).mean()
        
    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    # drop AdjustedClose column
    #feats = feats.drop([close_col], axis=1)

    return feats

In [68]:
# generate feature
buff = []
for code in tqdm(codes):
    feat = get_features_for_predict(df_price, code)
    buff.append(feat)
price_feature = pd.concat(buff)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [69]:
def adjust_financial(df_financial):
    # Drop invalid row
    df_financial = df_financial[~df_financial['DisclosedTime'].isna()]
    
    # Extract only consolidated financial reports
    df_financial = df_financial[df_financial['TypeOfDocument'].str.contains('FinancialStatements_Consolidated')]
    # If disclosure time >= 15:00, we can't use the information on the day
    df_financial['DisclosedDateTime'] = pd.to_datetime(df_financial['DisclosedUnixTime'], unit = 's')
    df_financial['DisclosedDateTime'] = df_financial['DisclosedDateTime'].dt.tz_localize('utc').dt.tz_convert('Asia/Tokyo')
    df_financial['Date'] = df_financial['Date'] + (df_financial['DisclosedDateTime'].dt.hour >= 15) * timedelta(days = 1)
    
    
    # Convert EPS and so on to float
    df_financial['EarningsPerShare'] = pd.to_numeric(df_financial["EarningsPerShare"], errors = 'coerce').fillna(0)
    df_financial['ForecastEarningsPerShare'] = pd.to_numeric(df_financial["ForecastEarningsPerShare"], errors = 'coerce').fillna(0)
    df_financial['NetSales'] = pd.to_numeric(df_financial["NetSales"], errors = 'coerce').fillna(0)
    df_financial['Profit'] = pd.to_numeric(df_financial["Profit"], errors = 'coerce').fillna(0)
    df_financial['ForecastProfit'] = pd.to_numeric(df_financial["ForecastProfit"], errors = 'coerce').fillna(0)
    df_financial['EquityToAssetRatio'] = pd.to_numeric(df_financial["EquityToAssetRatio"], errors = 'coerce').fillna(0)
    df_financial['ForecastDividendPerShareAnnual'] = pd.to_numeric(df_financial["EquityToAssetRatio"], errors = 'coerce').fillna(0)
    return df_financial[['Date', 'SecuritiesCode', 'EarningsPerShare', 'ForecastEarningsPerShare', 'NetSales', 'Profit', 'ForecastProfit', 'EquityToAssetRatio', 'ForecastDividendPerShareAnnual']]

In [70]:
df_financial =  pd.read_csv(f"{path}/financials.csv")
df_financial['Date'] = pd.to_datetime(df_financial['Date'],format="%Y-%m-%d")
df_financial = adjust_financial(df_financial)
df_financial

,Date,SecuritiesCode,EarningsPerShare,ForecastEarningsPerShare,NetSales,Profit,ForecastProfit,EquityToAssetRatio,ForecastDividendPerShareAnnual
0,2017-01-04,2753.0,218.23,319.76,2.276100e+10,1.494000e+09,2.190000e+09,0.817,0.817
1,2017-01-05,3353.0,328.57,485.36,2.212800e+10,6.290000e+08,9.300000e+08,0.301,0.301
3,2017-01-06,2659.0,224.35,285.05,1.347810e+11,7.171000e+09,9.111000e+09,0.765,0.765
7,2017-01-06,3344.0,-120.01,47.70,5.445200e+10,-6.690000e+08,2.500000e+08,0.266,0.266
9,2017-01-06,7453.0,753.78,913.28,2.470270e+11,1.999600e+10,2.430000e+10,0.703,0.703
...,...,...,...,...,...,...,...,...,...
92950,2021-12-04,4707.0,5.56,35.53,2.545000e+09,3.100000e+07,1.990000e+08,0.487,0.487
92951,2021-12-04,6040.0,-13.59,0.00,7.320000e+08,-2.060000e+08,0.000000e+00,0.653,0.653
92952,2021-12-04,6898.0,184.73,142.01,1.293000e+09,1.210000e+08,9.300000e+07,0.774,0.774
92954,2021-12-04,8057.0,153.74,330.92,4.307100e+10,1.507000e+09,3.250000e+09,0.396,0.396


In [71]:
def create_financial_features(df_financial):
    df = df_financial.copy()
    
    features = df[['Date', 'SecuritiesCode', 'EarningsPerShare', 'ForecastEarningsPerShare', 'NetSales', 'Profit', 'ForecastProfit', 'EquityToAssetRatio', 'ForecastDividendPerShareAnnual']]
    features['eps'] = features['EarningsPerShare']
    features['feps'] = features['ForecastEarningsPerShare']
    features['eps_feps'] = (features['eps'] / features['feps']).replace([np.inf, -np.inf], np.nan).fillna(0)
    features['pmargin'] = (features['Profit'] / features['NetSales']).replace([np.inf, -np.inf], np.nan).fillna(0)
    features['earatio'] = features['EquityToAssetRatio']
    features['fdps'] = features['ForecastDividendPerShareAnnual']
    

    return features.drop(['EarningsPerShare', 'ForecastEarningsPerShare', 'NetSales', 'Profit', 'ForecastProfit', 'EquityToAssetRatio', 'ForecastDividendPerShareAnnual'], axis = 1)


In [72]:
fin_features = create_financial_features(df_financial)
fin_features

,Date,SecuritiesCode,eps,feps,eps_feps,pmargin,earatio,fdps
0,2017-01-04,2753.0,218.23,319.76,0.682481,0.065639,0.817,0.817
1,2017-01-05,3353.0,328.57,485.36,0.676961,0.028426,0.301,0.301
3,2017-01-06,2659.0,224.35,285.05,0.787055,0.053205,0.765,0.765
7,2017-01-06,3344.0,-120.01,47.70,-2.515933,-0.012286,0.266,0.266
9,2017-01-06,7453.0,753.78,913.28,0.825355,0.080947,0.703,0.703
...,...,...,...,...,...,...,...,...
92950,2021-12-04,4707.0,5.56,35.53,0.156487,0.012181,0.487,0.487
92951,2021-12-04,6040.0,-13.59,0.00,0.000000,-0.281421,0.653,0.653
92952,2021-12-04,6898.0,184.73,142.01,1.300824,0.093581,0.774,0.774
92954,2021-12-04,8057.0,153.74,330.92,0.464584,0.034989,0.396,0.396


In [73]:
full_features = price_feature.merge(fin_features, on = ['Date', 'SecuritiesCode'], how = 'left')
for column_name in ['eps', 'feps', 'eps_feps', 'pmargin', 'earatio', 'fdps']:
    #full_features[column_name].ffill(inplace = True)
    full_features[column_name].fillna(0, inplace = True)

full_features

,Date,SecuritiesCode,Target,AdjustedClose,AdjustedVolume,return_2week,return_1month,return_3month,volatility_2week,volatility_1month,volatility_3month,feature_RSI_10days,feature_RSI_21days,feature_RSI_63days,rsi10,rsi21,rsi63,EMA10days,EMA21days,EMA63days,eps,feps,eps_feps,pmargin,earatio,fdps
0,2017-01-04,1301,0.000730,2742.0,31400.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.000000,2742.000000,2742.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-05,1301,0.002920,2738.0,17900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2741.272727,2741.636364,2741.875000,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-06,1301,-0.001092,2740.0,19900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2741.041322,2741.487603,2741.816406,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-10,1301,-0.005100,2748.0,24200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.306536,2742.079639,2742.009644,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-11,1301,-0.003295,2745.0,9300.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.796257,2742.345127,2742.103092,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332529,2021-11-29,9997,0.026987,668.0,320800.0,-0.116402,-0.170186,-0.176326,0.012748,0.012622,0.013956,12.457561,21.970251,35.743688,low,low,low,709.682425,737.486810,800.367258,0.0,0.0,0.0,0.0,0.0,0.0
2332530,2021-11-30,9997,-0.001460,667.0,296300.0,-0.107095,-0.171429,-0.176543,0.013207,0.012573,0.013955,12.315605,21.851355,35.692354,low,low,low,701.921984,731.078918,796.199531,0.0,0.0,0.0,0.0,0.0,0.0
2332531,2021-12-01,9997,0.017544,685.0,339100.0,-0.062927,-0.141604,-0.171705,0.017227,0.014779,0.014135,28.590144,29.102748,37.338325,low,low,low,698.845260,726.889926,792.724546,0.0,0.0,0.0,0.0,0.0,0.0
2332532,2021-12-02,9997,0.014368,684.0,342900.0,-0.068120,-0.117419,-0.167883,0.016934,0.013932,0.014131,28.266296,28.946071,37.284453,low,low,low,696.146122,722.990842,789.326904,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
data = full_features

In [51]:
targets_pivot = df_price.reset_index().pivot(index='Date',columns='SecuritiesCode',values='Target')
targets_pivot

SecuritiesCode      1301      1332      1333      1375      1376      1377  \
Date                                                                         
2017-01-04      0.000730  0.012324  0.006154       NaN  0.011053  0.003026   
2017-01-05      0.002920 -0.022609 -0.010703       NaN  0.005145  0.004525   
2017-01-06     -0.001092 -0.016014 -0.015456       NaN  0.000000 -0.033033   
2017-01-10     -0.005100 -0.016275 -0.001570       NaN -0.026871  0.046584   
2017-01-11     -0.003295  0.016544  0.015723       NaN  0.011177 -0.010386   
...                  ...       ...       ...       ...       ...       ...   
2021-11-29      0.003793  0.001745  0.009662  0.011504  0.003709  0.003200   
2021-11-30      0.007558 -0.001742  0.001740 -0.015435 -0.019217 -0.007974   
2021-12-01      0.016706  0.020942  0.023882  0.018977  0.003768  0.019293   
2021-12-02     -0.003689  0.006838  0.008058 -0.008907  0.014264  0.009464   
2021-12-03      0.031639 -0.056027  0.013462  0.032680  0.032568  0.026562   

SecuritiesCode      1379      1381      1407      1413      1414      1417  \
Date                                                                         
2017-01-04      0.005169 -0.009326 -0.003437       NaN  0.000000 -0.007463   
2017-01-05     -0.008883  0.019874 -0.009006       NaN -0.013917 -0.002820   
2017-01-06     -0.005189 -0.007692 -0.019528       NaN  0.005040  0.007540   
2017-01-10     -0.007112  0.000000  0.004733       NaN -0.009027 -0.015903   
2017-01-11     -0.008118  0.001034  0.000000       NaN -0.003036  0.000951   
...                  ...       ...       ...       ...       ...       ...   
2021-11-29      0.025516  0.015949 -0.039695  0.002361  0.014330  0.003562   
2021-11-30     -0.011646 -0.010989 -0.034976 -0.005181 -0.003027 -0.004564   
2021-12-01      0.012855  0.015873  0.019769  0.040246  0.032389  0.013754   
2021-12-02     -0.001058 -0.015625  0.004847 -0.003186 -0.007843  0.000503   
2021-12-03      0.019058  0.004762  0.019293  0.027397  0.015810  0.015570   

SecuritiesCode      1419      1429      1431      1435      1515      1518  \
Date                                                                         
2017-01-04      0.009259  0.007160       NaN -0.023641 -0.026502 -0.007559   
2017-01-05      0.018349 -0.007109       NaN -0.033898  0.000000 -0.008378   
2017-01-06     -0.003604  0.002387       NaN -0.020050  0.029038  0.011521   
2017-01-10     -0.014467 -0.011905       NaN -0.017903 -0.010582 -0.020501   
2017-01-11      0.001835  0.004819       NaN  0.014323 -0.001783 -0.009302   
...                  ...       ...       ...       ...       ...       ...   
2021-11-29     -0.003057  0.004464  0.001225 -0.004525  0.014610  0.018561   
2021-11-30     -0.024967  0.014815 -0.009792 -0.031818 -0.009600  0.003797   
2021-12-01      0.026056  0.040876  0.025958  0.061033  0.025848  0.024962   
2021-12-02      0.001751 -0.001403  0.008434 -0.017699 -0.006299 -0.016974   
2021-12-03      0.029283  0.049157  0.033453  0.049550  0.015848  0.048799   

SecuritiesCode      1605      1662      1663      1712      1716      1719  \
Date                                                                         
2017-01-04     -0.012414 -0.004915  0.006963 -0.008059  0.004916 -0.005051   
2017-01-05     -0.015171 -0.017477 -0.020745  0.014218 -0.005871  0.007614   
2017-01-06      0.013644  0.003867 -0.011950  0.015381  0.013780 -0.011335   
2017-01-10     -0.005211 -0.000385 -0.039582  0.001341 -0.008738  0.008917   
2017-01-11      0.004365  0.015029 -0.006869  0.015049 -0.001959 -0.001263   
...                  ...       ...       ...       ...       ...       ...   
2021-11-29      0.001067  0.009603 -0.025965 -0.002413  0.002364  0.019025   
2021-11-30     -0.026652  0.006485  0.021189 -0.035067  0.000786  0.011669   
2021-12-01      0.015334  0.015893 -0.019411  0.041980  0.025923  0.016148   
2021-12-02      0.021575  0.013531 -0.003413 -0.009621  0.004594 -0.001135   
2021-12-03      0.0

In [53]:
target_rolling5 = df_price['Target'].rolling(5).agg(['mean','var'])
target_rolling5

,mean,var
Date,,
2017-01-04,NaN,NaN
2017-01-05,NaN,NaN
2017-01-06,NaN,NaN
2017-01-10,NaN,NaN
2017-01-11,-0.001167,0.000010
...,...,...
2021-11-29,-0.004962,0.000530
2021-11-30,-0.006107,0.000510
2021-12-01,0.002501,0.000464


In [76]:
df_price = df_price.reset_index()
params_garch = {}
means_garch = {}
stds_garch = []

for code, targets in tqdm(df_price['Target'].groupby('SecuritiesCode')):
    am = arch.arch_model(targets[targets.notna()]*100, p=1, o=1, q=1, power=1.0, dist="StudentsT")
    result = am.fit(update_freq=5, disp="off")
    params_garch[code] = result.params
    means_garch[code] = result.params.mu
    stds_garch.append(pd.DataFrame(result.conditional_volatility))

stds_garch = pd.concat(stds_garch)
display(stds_garch)

KeyError: 'SecuritiesCode'

In [78]:
#Set index-
data = data.set_index(['Date','SecuritiesCode'])
display(data)

#filter out codes without data
codes = data.index.get_level_values(1).unique()
display(codes)

,,Target,AdjustedClose,AdjustedVolume,return_2week,return_1month,return_3month,volatility_2week,volatility_1month,volatility_3month,feature_RSI_10days,feature_RSI_21days,feature_RSI_63days,rsi10,rsi21,rsi63,EMA10days,EMA21days,EMA63days,eps,feps,eps_feps,pmargin,earatio,fdps
Date,SecuritiesCode,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-04,1301,0.000730,2742.0,31400.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.000000,2742.000000,2742.000000,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-05,1301,0.002920,2738.0,17900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2741.272727,2741.636364,2741.875000,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-06,1301,-0.001092,2740.0,19900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2741.041322,2741.487603,2741.816406,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-10,1301,-0.005100,2748.0,24200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.306536,2742.079639,2742.009644,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-11,1301,-0.003295,2745.0,9300.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low,low,low,2742.796257,2742.345127,2742.103092,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-29,9997,0.026987,668.0,320800.0,-0.116402,-0.170186,-0.176326,0.012748,0.012622,0.013956,12.457561,21.970251,35.743688,low,low,low,709.682425,737.486810,800.367258,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-30,9997,-0.001460,667.0,296300.0,-0.107095,-0.171429,-0.176543,0.013207,0.012573,0.013955,12.315605,21.851355,35.692354,low,low,low,701.921984,731.078918,796.199531,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01,9997,0.017544,685.0,339100.0,-0.062927,-0.141604,-0.171705,0.017227,0.014779,0.014135,28.590144,29.102748,37.338325,low,low,low,698.845260,726.889926,792.724546,0.0,0.0,0.0,0.0,0.0,0.0


Int64Index([1301, 1332, 1333, 1375, 1376, 1377, 1379, 1381, 1407, 1413,
            ...
            9982, 9983, 9984, 9987, 9989, 9990, 9991, 9993, 9994, 9997],
           dtype='int64', name='SecuritiesCode', length=2000)

In [80]:
import tensorflow as tf

tf.random.set_seed(2022)
tf.debugging.set_log_device_placement(False)
try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
except ValueError:
    tpu_strategy = tf.distribute.get_strategy()
    print(f"Running on {tpu_strategy.num_replicas_in_sync} replicas")

Running on 1 replicas


In [ ]:
import tensorflow.keras.utils as ku
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl

def create_data():
    len_ts = 30
    len_valid = 90
    len_burn_start = 63
    len_burn_end = 5
    
    x_columns = ['AdjustedClose','AdjustedVolume','return_2week','return_1month','return_3month','volatility_2week','volatility_1month','volatility_3month','feature_RSI_10days','feature_RSI_21days','feature_RSI_63days','EMA10days','EMA21days','EMA63days','rsi10','rsi21','rsi63','eps','feps','eps_feps','pmargin','earatio','fdps']
    y_columns = ['Target']
    
    x_train = []
    y_train = []
    x_valid = []
    y_valid = []
    
    for code in tqdm(codes):
        data_code = data.xs(code, level=1)
        idx_train_start = len_burn_start
        idx_valid_end = len(data_code.index)-1-len_burn_end
        idx_valid_start = idx_valid_end-len_valid
        idx_train_end = idx_valid_start-len_burn_start
        num_iter_train = idx_train_end-idx_train_start-len_ts
        num_iter_valid = idx_valid_end-idx_valid_start-len_ts
        for i in range(num_iter_train):
            idx_start = idx_train_start+i
            idx_end = idx_start+len_ts
            x = data_code.iloc[idx_start:idx_end][x_columns].values
            y = data_code.iloc[idx_end][y_columns].values
            x_train.append(x)
            y_train.append(y)
        for i in range(num_iter_valid):
            idx_start = idx_valid_start+i
            idx_end = idx_start+len_ts
            x = data_code.iloc[idx_start:idx_end][x_columns].values
            y = data_code.iloc[idx_end][y_columns].values
            x_valid.append(x)
            y_valid.append(y)
    x_train = np.array(x_train).astype(np.float32)
    y_train = np.array(y_train).astype(np.float32)
    x_valid = np.array(x_valid).astype(np.float32)
    y_valid = np.array(y_valid).astype(np.float32)
    x_shape = x_train.shape[-2:]
    y_shape = y_train.shape[-1]
    return x_shape,y_shape, x_train,y_train,x_valid,y_valid

def create_model(x_shape,y_shape):
    i = kl.Input(shape=x_shape)
    l1 = kl.LSTM(units=512, return_sequences=True)(i)
    l2 = kl.GlobalMaxPooling1D()(l1)
    o = kl.Dense(y_shape, activation='sigmoid')(l2)
    return km.Model(inputs=i, outputs=o)

x_shape,y_shape, x_train,y_train,x_valid,y_valid = create_data()
display(x_shape,y_shape)
model = create_model(x_shape, y_shape)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.saved_model import SaveOptions, LoadOptions

with tpu_strategy.scope():  
    EPOCHS=30
    BATCH_SIZE=256
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    x_train = x_train.reshape([-1,x_shape[-1]])
    x_train = x_scaler.fit_transform(x_train)
    y_train = y_scaler.fit_transform(y_train)
    x_train = x_train.reshape([-1,x_shape[-2],x_shape[-1]])
    
    x_valid = x_valid.reshape([-1,x_shape[-1]])
    x_valid = x_scaler.transform(x_valid)
    x_valid = x_valid.reshape([-1,x_shape[-2],x_shape[-1]])
    y_valid = y_scaler.transform(y_valid)
    
    
    model.compile(optimizer=Adam(0.001), loss='MSE', metrics='MSE')
    lr = ReduceLROnPlateau(monitor='val_MSE', mode='max', factor=0.5, patience=1, verbose=False)
    es = EarlyStopping(monitor='val_MSE', mode='max', patience=5, restore_best_weights=True, verbose=False)
    cp = ModelCheckpoint(f'./TPS_model.h5', monitor='val_MSE', mode='max', save_best_only=True, options=SaveOptions(experimental_io_device='job:localhost'))
    history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=EPOCHS, batch_size=BATCH_SIZE,  callbacks=[lr, es, cp])

    pp.plot(history.history['MSE'])
    pp.plot(history.history['val_MSE'])